Importação das Bibliotecas Necessárias

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Camada Bronze: Carregamento dos Dados Brutos

In [ ]:
# Carregando os dados brutos a partir do arquivo CSV
df_bronze = pd.read_csv('dataset/credit_risk_dataset.csv')

# Visualizando as primeiras linhas do DataFrame
print(df_bronze.head())

Camada Silver: Limpeza e Transformação Básica

In [ ]:
# Criando uma cópia dos dados brutos
df_silver = df_bronze.copy()

# Verificando valores ausentes
print(df_silver.isnull().sum())

# Tratamento de valores ausentes (se houver)
# Exemplo: Preencher valores ausentes em 'person_emp_length' com a mediana
if df_silver['person_emp_length'].isnull().sum() > 0:
    df_silver['person_emp_length'].fillna(df_silver['person_emp_length'].median(), inplace=True)

# Convertendo tipos de dados
df_silver['person_age'] = df_silver['person_age'].astype(int)
df_silver['person_income'] = df_silver['person_income'].astype(float)
df_silver['loan_amnt'] = df_silver['loan_amnt'].astype(float)
df_silver['loan_int_rate'] = df_silver['loan_int_rate'].astype(float)
df_silver['loan_status'] = df_silver['loan_status'].astype(int)
df_silver['loan_percent_income'] = df_silver['loan_percent_income'].astype(float)
df_silver['cb_person_cred_hist_length'] = df_silver['cb_person_cred_hist_length'].astype(int)


Camada Gold: Enriquecimento e Preparação para ML

In [ ]:
# Criando uma cópia dos dados da camada silver
df_gold = df_silver.copy()

# Engenharia de Características

# Criar uma variável binária para indicar se a pessoa possui casa própria
df_gold['is_home_owner'] = df_gold['person_home_ownership'].apply(lambda x: 1 if x in ['OWN', 'MORTGAGE'] else 0)

# Codificação de variáveis categóricas
categorical_vars = ['loan_intent', 'loan_grade', 'cb_person_default_on_file']

# Usando one-hot encoding para variáveis categóricas
df_gold = pd.get_dummies(df_gold, columns=categorical_vars, drop_first=True)

# Atualizando a lista de features com as novas colunas criadas
features = [
    'person_age',
    'person_income',
    'person_emp_length',
    'loan_amnt',
    'loan_int_rate',
    'loan_percent_income',
    'cb_person_cred_hist_length',
    'is_home_owner'
]

# Adicionando as colunas de variáveis dummies
features += [col for col in df_gold.columns if col.startswith('loan_intent_') or col.startswith('loan_grade_') or col.startswith('cb_person_default_on_file_')]

# Definindo as variáveis X e y
X = df_gold[features]
y = df_gold['loan_status']
